# building train data

-for a line, extract features based on this line and previous line, label=1

-for a line, extract features based on this line and a random line, label=0

-generate a large number of sents, for a low loglik generated line (bottom ranked K in the normalized-loglik list), 

-extract features for this line and a random line from the corpus, label=0

-do we need to generate a lot of sentences for the negative examples?

-we could also impose some hard coded constraints on the loglik threshold.

In [3]:
f=open('rank/test/lyrics_test_data_clean.txt','r').read().split('\n\n')
print len(f)

11893


In [7]:
for i in f[:3]:
    print i
    print '---------'


But don't leave your keys for me, open the door
Nobody's getting hurt here if nobody cares
Nobody's getting lost if nobody's going anywhere
---------
And darling I know that you can't be sure of anything anymore
---------
I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll never let you go
---------


In [ ]:
import kenlm
import gensim,os
import numpy as np
from scipy.spatial.distance import cosine
from gensim.models import Doc2Vec
import pronouncing



def is_rhyme(word1,word2):
    word1=re.sub(u'[^A-Za-z]','',word1)
    word2=re.sub(u'[^A-Za-z]','',word2)
    return int(word1 in pronouncing.rhymes(word2))
    
def get_loglik_norm(sent,LM):
    model = kenlm.LanguageModel(LM)
    loglik_norm=model.score(sent)/len(sent.split(' '))
    return loglik_norm

def get_d2v_dist(sent1,sent2,model):
    model = Doc2Vec.load(model)
    a=model.infer_vector(sent1.split(' '))
    b=model.infer_vector(sent2.split(' '))
    return cosine(a,b)


def make_feature_vec(words, model, num_features):
    # average all of the word vectors in a sentence
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def get_w2v_dist(sent1,sent2,model):
    v1=make_feature_vec(sent1,model,100)
    v2=make_feature_vec(sent2,model,100)
    return cosine(v1,v2)


def extract_features_positive(passage,LM,w2v_model,d2v_model):
    """extract feature from one passage"""
    # a passage is a consecutive set of lines without a blank line in between. we extract features with these pairs 
    # of lines as prev and next lines. they're a more coherent unit. The passages is obtained by methods above, 
    # namely, splitting the training file by '\n\n'
    line_list=passage.split('\n')
    line_list=[i for i in line_list if i!='']
    features=['loglik_norm','d2v_dist','w2v_dist','rhyme_prev','rhyme_current','num_syl_cur','num_syl_prev','class']
    pos_feature_vec=[]
    for i in range(1,len(line_list)):
        #extract features from the current and prev line
        prev=line_list[i-1]
        current=line_list[i]
        #features
        loglik_norm=get_loglik_norm(current,LM)#LM='train3.lm'
        d2v_dist=get_d2v_dist(prev,current,d2v_model)
        w2v_dist=get_w2v_dist(prev,current,w2v_model)
        rhyme_prev=is_rhyme(prev.split(' ')[-1])
        
        
        
        
        


In [11]:
line_list

['',
 "But don't leave your keys for me, open the door",
 "Nobody's getting hurt here if nobody cares",
 "Nobody's getting lost if nobody's going anywhere"]

In [ ]:
d2v_model='rock_train.d2v'
w2v_model='rock_train.w2v'